### **SỬ DỤNG MÔ HÌNH HỒI QUY (REGRESSION) ĐỂ DỰ ĐOÁN DÂN SỐ CÁC NƯỚC TRONG TƯƠNG LAI**

<hr />

## **Import Thư viện**

Mô hình này cần dữ liệu để phân tích và vẽ đồ thị trực quan để đưa ra hướng đi hợp lí để tiến hành dự đoán.

In [12]:
#from bokeh.plotting.figure import figure
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
df=pd.read_csv('./population.csv')


## **Hồi quy Logistic các yếu tố ảnh hưởng đến dân số. Đâu là đất nước đáng sống?**

Dân số là một trong những yếu tố quyết định chất lượng sống của 1 quốc gia, vấn đề kiểm soát dân số cũng là một vấn đề quan trọng hàng đầu. Chúng ta hãy cũng quan sát những nước được xem là ***Đáng sống nhất*** thế giới có những chỉ số như thế nào trong việc kiểm soát dân số.

Một cuộc khảo sát với khoảng hơn 17.000 người từ khắp thế giới tham gia trên trang chuyên đưa ra các phân tích và xếp hạng **"www.usnews.com"** về vấn đề **"Nước nào đáng sống nhất"**. Chúng ta sẽ lấy thông tin của top 50 nước hàng đầu và lưu vào dataframe ***df_train*** bằng html request đơn giản.

In [13]:
import requests 

headers = {'User-Agent': 'Mozilla/5.0'}
r = requests.get('https://www.usnews.com/news/best-countries/rankings/quality-of-life?renderer=json',headers=headers)
best_country_to_live=r.json()
best_country_to_live=[i['country'] for i in best_country_to_live['rankings']][:50]
df_train=df[df['Year']==2020]
df_train=df_train[['Country','Population','Yearly%Change','MedianAge','UrbanPop%','Migrants(net)','FertilityRate']]
df_train['BestToLive']=[False]*df_train.shape[0]
df_train=df_train.reset_index(drop=True)
for i in best_country_to_live:
    try:
        index = df_train[df_train['Country']==i].index[0]
        df_train.at[index,'BestToLive']=True
    except:
        pass
df_train.head()

,Country,Population,Yearly%Change,MedianAge,UrbanPop%,Migrants(net),FertilityRate,BestToLive
0,Afghanistan,38928346,2.33,18.4,25.4,-62920.0,4.56,False
1,Albania,2877797,-0.11,36.4,63.5,-14000.0,1.62,False
2,Algeria,43851044,1.85,28.5,72.9,-10000.0,3.05,False
3,American Samoa,55191,-0.22,0.0,88.1,0.0,0.00,False
4,Andorra,77265,-0.19,0.0,87.8,0.0,0.00,False


Ta sẽ lấy dữ liệu ở năm 2020 ra để tiến hành *phân lớp*. **Tại sao lại là 2020?**, là để sau khi tiến hành dự đoán, ta thử lấy kết quả đối chiếu với thời gian hiện tại, là vào cuối năm 2022 để đánh giá mô hình.

Lựa chọn phân lớp theo 2 thuộc tính (đơn giản hóa bài toán dễ tạo nhiều mô hình).

**1)** Một đất nước phát triển, thì mật độ dân số ở khu vực thành thị sẽ rất cao, cũng đồng nghĩa với việc sẽ có nhiều dân cư ở nước khác sinh sống và làm việc tại các đất nước này. Vì vậy đầu tiên ta sẽ phân lớp và quan sát trên 2 thuộc tính: **UrbanPop%** (Tỉ lệ dân số thành thị) và **Migrants(net)** (Số dân nhập cư).

In [14]:
X = [ [df_train.iloc[i]['UrbanPop%'], df_train.iloc[i]['Migrants(net)']] for i in range(df_train.shape[0]) ]
y= list(df_train['BestToLive'])

Vẽ đồ thị trực quan với trục hoành là trục UrbanPop% và trục tung là Migrants(net), và điểm chấm màu đỏ là các nước được cho là **Đáng sống**.

In [15]:
fig=px.scatter(df_train, x="UrbanPop%", y="Migrants(net)",
        color="BestToLive", hover_data=['Country'])
fig.update_layout(
    title={
        'text': "Trực quan giữa UrbanPop% và Migrants(net)",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

Với tập train có cột label được xem là có đáng sống hay không? (True hoặc False), ta xây dựng mô hình **Hồi quy logistic** để đưa ra dự đoán dựa trên 2 yếu tố UrbanPop% và Migrants.

In [16]:


import random
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs

from sklearn.metrics import log_loss

X = [ [df_train.iloc[i]['UrbanPop%'], df_train.iloc[i]['Migrants(net)']] for i in range(df_train.shape[0]) ]
y= list(df_train['BestToLive'])

model = LogisticRegression()
model.fit(X, y)

#kiểm tra
#Xnew = [[80,500 * 1000],[40,100],[50,-100]]
#Xnew=[[random.randint(20,100),10**(random.randint(0,60)/10)] for i in range(500)]
Xnew=X
ynew = model.predict_proba(Xnew)

dict_={
    'Country' : [i for i in df_train['Country']],
    'UrbanPop%': [i[0] for i in Xnew],
    'Migrants(net)': [i[1] for i in Xnew],
    'predict':[i[1] for i in ynew]
}
ynew = model.predict(Xnew)

print('Binary cross entropy loss là : ',log_loss(y, ynew))
print('Độ chính xác của mô hình : ',sum(ynew==y)/len(y))

Binary cross entropy loss là :  6.466844936886641
Độ chính xác của mô hình :  0.8127659574468085


In [17]:
fig=px.scatter(dict_, x="UrbanPop%", y="Migrants(net)",
        color="predict", hover_data=['Country'], color_continuous_scale=['green','orange', 'purple'], trendline='ols')

fig.update_layout(
    title={
        'text': "Kiểm tra bài toán phân lớp bằng thuộc tính UrbanPop% và Migrants(net)",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

**Đánh giá**: Như vậy, theo mô hình thì dân số thành thị cao nhưng số dân nhập cư tới thấp thì dự đoán sẽ rất thấp.

Có vẻ hợp lý khi kết hợp 2 yếu tố này ngoài thực tế thì dự đoán lại rất cao, còn không dự đoán sẽ lệch về phía nữa trên đồ thị và có xu hướng lệch nhẹ về bên trái. Với độ chính xác trên tập train là khoảng 0.81 thì lượng dân nhập cư lớn sẽ phản ánh phần nào mức độ đáng sống của 1 quốc gia. Vì đơn giản ***Ai cũng muốn đến đây sống***

**2)** 2 yếu tố có thể được coi là quan trọng nhất trong công cuộc kiểm soát dân số của một quốc gia đó chính là ***Tỉ lệ sinh*** và ***Tỉ lệ tử***. Một đất nước được cho là phát triển khi có thể áp dụng các chính sách kiểm soát được 2 tỉ lệ này một cách hiệu quả.

Trong dữ liệu thu thập, chỉ có cột ***Tỉ lệ sinh (FertilityRate)***, vì thế ta có thể dùng cột ***Tỉ lệ thay đổi dân số (Yearly%Change)*** để tiến hành mô hình hoá dữ liệu. Trong đó chấm đỏ là các đất nước được xem là đáng sống.

In [18]:
fig=px.scatter(df_train[df_train['FertilityRate']>0], x="Yearly%Change", y="FertilityRate",
        color="BestToLive", hover_data=['Country'])
fig.update_layout(
    title={
        'text': "Trực quan giữa Yearly%Change và FertilityRate",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

Mô hình Hồi quy Logistic cho 2 thuộc tính Yearly%Change và FertilityRate.

In [19]:

import random


from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs

X = [ [df_train[df_train['FertilityRate']>0].iloc[i]['Yearly%Change'], df_train[df_train['FertilityRate']>0].iloc[i]['FertilityRate']] 
                for i in range(df_train[df_train['FertilityRate']>0].shape[0]) ]
y= list(df_train[df_train['FertilityRate']>0]['BestToLive'])

model = LogisticRegression()
model.fit(X, y)

#Kiểm tra 
Xnew=X
ynew = model.predict_proba(Xnew)

country_list = [df_train[df_train['FertilityRate']>0].iloc[i]['Country'] for i in range(df_train[df_train['FertilityRate']>0].shape[0])]
dict_={
    'Country' : [j for j in country_list],
    'Yearly%Change': [i[0] for i in Xnew],
    'FertilityRate': [i[1] for i in Xnew],
    'predict':[i[1] for i in ynew]
}
ynew = model.predict(Xnew)
print('Binary cross entropy loss là : ',log_loss(y, ynew))
print('Độ chính xác của mô hình : ',sum(ynew==y)/len(y))


Binary cross entropy loss là :  8.763605932921974
Độ chính xác của mô hình :  0.746268656716418


In [20]:
fig=px.scatter(dict_, x="Yearly%Change", y="FertilityRate",
        color="predict", hover_data=['Country'], color_continuous_scale=['green','orange', 'purple'],trendline='ols')

# fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Regression Fit'))
fig.update_layout(
    title={
        'text': "Kiểm tra bài toán phân lớp bằng thuộc tính FertilityRate và Yearly%Change",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

**Đánh giá**: Có vẻ như tỉ lệ sinh và số lượng dân số không thay đổi không quyết định nhiều lắm về chất lượng quốc gia đáng sống.

## **BÀI TOÁN DỰ ĐOÁN DÂN SỐ TƯƠNG LAI, HỒI QUY TUYẾN TÍNH.**

Hồi quy tuyến tính là một phương pháp rất đơn giản nhưng đã được chứng minh được tính hữu ích cho một số lượng lớn các tình huống, là một phương pháp thống kê để hồi quy dữ liệu với biến phụ thuộc có giá trị liên tục trong khi các biến độc lập có thể có một trong hai giá trị liên tục hoặc là giá trị phân loại. Trong bài toán dân số này, biến độc lập chính là năm (Year) và biến phụ thuộc chính là dân số (Population)

Ở bài này ta sẽ sử dụng mô hình Hồi quy tuyến tính (Linear Regression) và Hồi quy đa thức (Polynomial Regression) bậc 2. Trong đó Hồi quy tuyến tính có thể xem là Hồi quy đa thức bậc 1.

### Xây dựng hàm

Nói chung là khá đơn giản, vì thư viện sklearn đã có sẵn model LinearRegression. Ta chỉ cần dùng thêm hàm PolynomialFeatures để đa thức hoá dữ liệu đầu vào và xây dựng hàm mất mát cho bài toán **pred_error**.

In [21]:
import numpy as np
import math 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
def linear_regression_1(x,y,x_for_pred) :
    x=np.array(x).reshape((-1, 1))
    y=np.array(y)
    x_for_pred=np.array(x_for_pred).reshape((-1, 1))
    model = LinearRegression().fit(x, y)
    y_pred = model.predict(x_for_pred)

    y_pre_pred = model.predict(x)
    pred_error=math.sqrt(mean_squared_error(y,y_pre_pred))/math.sqrt((max(y_pre_pred)-min(y_pre_pred))**2 +(max(x)-min(x))**2)
    return y_pred, y_pre_pred, pred_error
def linear_regression_2(x,y,x_for_pred) :
    x=np.array(x)
    x_=PolynomialFeatures(degree=2, include_bias=False).fit_transform(x.reshape((-1, 1)))
    y=np.array(y)
    x_for_pred=np.array(x_for_pred).reshape((-1, 1))
    x_for_pred=PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_for_pred)
    
    model = LinearRegression().fit(x_, y)
    
    y_pred = model.predict( x_for_pred)
    y_pre_pred = model.predict(x_)
    pred_error=math.sqrt(mean_squared_error(y,y_pre_pred))/math.sqrt((max(y_pre_pred)-min(y_pre_pred))**2 +(max(x)-min(x))**2)
    return y_pred, y_pre_pred, pred_error




Ta sẽ tiến hành kiểm tra hàm xây dựng được qua tình hình dân số nước **Afghanistan**.

**Tuyến tính (Bậc 1)**

In [22]:
y_pred, y_pre_pred, pred_error = linear_regression_1(df['Year'].unique(),df[df['Country']=='Afghanistan']['Population'],[2022,2025,2030,2035,2040,2050])
pop=np.array(df['Year'].unique())
china=df[df['Country']=='Afghanistan']['Population']

fig=px.scatter(x=pop, y=china) 
fig.add_traces(go.Scatter(x=pop, y= y_pre_pred, mode='lines', name='Regression Fit'))
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Population",
    title={
        'text': "Kiểm tra bài toán Hồi quy đa thức bậc 1 số lượng dân số của Afghanistanc",
        'x' : 'Year',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

**Đa thức bậc 2**

In [23]:
y_pred, y_pre_pred, pred_error = linear_regression_2(df['Year'].unique(),df[df['Country']=='Afghanistan']['Population'],[2025,2030,2035,2040,2050])
pop=np.array(df['Year'].unique())
china=df[df['Country']=='Afghanistan']['Population']

fig=px.scatter(x=pop, y=china) 
fig.add_traces(go.Scatter(x=pop, y= y_pre_pred, mode='lines', name='Regression Fit'))
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Population",
    title={
        'text': "Kiểm tra bài toán Hồi quy đa thức bậc 2 số lượng dân số của Afghanistan",
        'x' : 'Year',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

Có vẻ như hàm xây dựng đã đúng, ta tiến hành áp dụng mô hình tuyến tính cho tệp dữ liệu của 275 quốc gia. Và sau đó, đưa ra dự đoán dân số vào các năm 2022, 2025, 2030, 2035, 2040, 2050.

Sau khi đưa ra dự đoán, ta sẽ dựa vào hàm mất mát xây dựng được để đánh giá mô hình.

In [24]:
def gen_linear(df,funct,x_for_pred):
    x=df['Year'].unique()
    for i in df['Country'].unique():
    
        mean_pred_error=0
        pred_error=[]
        df_=df[df['Country']==i]
        dict_pred={}
        dict_pred['Year']=x_for_pred
        col_pred=['Population','YearlyChange','Migrants(net)','MedianAge','FertilityRate','UrbanPopulation']
        error_linear=0
        for col in col_pred : 
            dict_pred[col], pre_pred, error_linear=funct(x,df_[col],x_for_pred)
            pred_error.append(error_linear)
        dict_pred['Yearly%Change']=dict_pred['YearlyChange']*100/dict_pred['Population']
        dict_pred['UrbanPop%']=dict_pred['UrbanPopulation']*100/dict_pred['Population']
        dict_pred['Country']=[i]*len(x_for_pred)
        dict_pred['Continent']=list(df_['Continent'].unique())*len(x_for_pred)
        mean_pred_error=sum(pred_error)/len(pred_error)
        dict_pred['LossFunctCheck']=[mean_pred_error]*len(x_for_pred)
        
        #Có những nước với dụ đoán GIẢM có nguy cơ diệt vong (0 dân)
        dict_pred['Population']=[i if i>0 else 1000 for i in dict_pred.pop('Population')]
        dict_pred = [dict(zip(dict_pred,t)) for t in zip(*dict_pred.values())]
        for row in dict_pred :
            yield row


LD1=[i for i in gen_linear(df,linear_regression_1,[2022,2025,2030,2035,2040,2050]) ]
LD1=pd.DataFrame(LD1)
LD2=[i for i in gen_linear(df,linear_regression_2,[2022,2025,2030,2035,2040,2050]) ]
LD2=pd.DataFrame(LD2)

### **Trực quan hoá mô hình Hồi quy tuyến tính**

Vậy là ta đã có những dữ đoán cho tình hình dân số tương lai, liệu mô hình đã hiệu quã chưa? Ta cùng đánh giá qua hành mất mát.

**Hồi quy tuyến tính**

In [25]:
LD1

,Year,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck,Population
0,2022,9.697865e+05,-46311.662840,16.639918,5.529233,9.065679e+06,2.672341,24.981361,Afghanistan,Asia,1.482467,3.628977e+07
1,2025,1.015350e+06,-44993.213325,16.582543,5.409869,9.517517e+06,2.686857,25.185605,Afghanistan,Asia,1.482467,3.778951e+07
2,2030,1.091289e+06,-42795.797468,16.486920,5.210928,1.027058e+07,2.708648,25.492221,Afghanistan,Asia,1.482467,4.028908e+07
3,2035,1.167229e+06,-40598.381611,16.391296,5.011988,1.102364e+07,2.727894,25.763014,Afghanistan,Asia,1.482467,4.278864e+07
4,2040,1.243168e+06,-38400.965754,16.295673,4.813048,1.177671e+07,2.745014,26.003915,Afghanistan,Asia,1.482467,4.528821e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2025,2.038575e+05,-134017.823668,18.685330,3.276979,6.055059e+06,1.284438,38.150902,Zimbabwe,Africa,0.278630,1.587134e+07
1406,2030,2.075294e+05,-145549.966245,18.885654,2.957300,6.500572e+06,1.236126,38.719944,Zimbabwe,Africa,0.278630,1.678869e+07
1407,2035,2.112013e+05,-157082.108822,19.085978,2.637620,6.946084e+06,1.192821,39.230023,Zimbabwe,Africa,0.278630,1.770604e+07
1408,2040,2.148732e+05,-168614.251398,19.286302,2.317941,7.391596e+06,1.153782,39.689850,Zimbabwe,Africa,0.278630,1.862339e+07


In [26]:
fig=px.scatter(LD1[LD1['Year']==2050], x="Country", y="LossFunctCheck",range_y=[0,5],
        color="Continent")

fig.update_xaxes(visible=False)
fig.show()

**Nhận xét**

Điểm trừ của Linear bậc 1 : biên độ mất mát bị lệch do ảnh hướng từ các số liệu lệch đột biến. Ví dụ giả sử Afghanistan có dân số xu hướng tăng theo thời gian nhưng dân số năm 2022 và 2025 được dự đoán sẽ nhỏ hơn dân số năm 2020.

**Hồi quy bậc 2**

In [27]:
LD2

,Year,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck,Population
0,2022,1.068893e+06,3977.750578,17.953049,4.730352,1.042072e+07,2.589791,25.248080,Afghanistan,Asia,0.436912,4.127333e+07
1,2025,1.152376e+06,24537.823051,18.398103,4.305322,1.139103e+07,2.579184,25.494761,Afghanistan,Asia,0.436912,4.467987e+07
2,2030,1.299116e+06,62661.154521,19.240558,3.535674,1.311211e+07,2.560358,25.841964,Afghanistan,Asia,0.436912,5.073961e+07
3,2035,1.455355e+06,105605.166009,20.208889,2.689447,1.496309e+07,2.540903,26.124051,Afghanistan,Asia,0.436912,5.727707e+07
4,2040,1.621095e+06,153369.857515,21.303095,1.766641,1.694396e+07,2.521447,26.354597,Afghanistan,Asia,0.436912,6.429225e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2025,1.672271e+05,-155737.231362,19.810886,3.140785,6.271365e+06,1.080287,40.513016,Zimbabwe,Africa,0.161457,1.547988e+07
1406,2030,1.519725e+05,-178491.551093,20.592771,2.750736,6.828640e+06,0.938394,42.165210,Zimbabwe,Africa,0.161457,1.619496e+07
1407,2035,1.341783e+05,-202751.706494,21.452693,2.351245,7.400911e+06,0.794758,43.836712,Zimbabwe,Africa,0.161457,1.688291e+07
1408,2040,1.138444e+05,-228517.697563,22.390652,1.942311,7.988179e+06,0.648919,45.533010,Zimbabwe,Africa,0.161457,1.754371e+07


In [28]:
fig=px.scatter(LD2[LD2['Year']==2050], x="Country", y="LossFunctCheck",range_y=[0,5],
        color="Continent")

fig.update_xaxes(visible=False)
fig.show()

**Nhận xét**

Có vẻ dân số sẽ tăng đều theo xu hướng của biểu đồ?

Điểm trừ của LD2 : Overfit, không phù hợp để chuẩn đoán các mốc thời gian xa hơn, vì ta dự đoán tới năm 2050. Tuy nhiên rất tốt để chuẩn đoán các giá trị liền kề, nhất là năm 2022 (Hiện tại) từ dữ liệu tới năm 2020.

### **Đưa ra giải pháp tối ưu mô hình**

Giả thuyết về độ mất mát lớn: Dữ liệu được thu thập vào năm 1950, là năm hậu chiến tranh thế giới thứ 2 và vẫn còn diễn ra xung đột vũ trang ở một số quốc gia và vùng lãnh thổ, dẫn đến biến động về dân số.

Vì thế ta đặt mốc ổn định của thế giới là năm 1990 (tình hình thế giới có thể được coi là ít xung độ vũ trang) để cải tiến dự đoán của mô hình Hồi quy Tuyến tính.

In [29]:
LD3=[i for i in gen_linear(df[df['Year']>=1990],linear_regression_1,[2025,2030,2035,2040,2050]) ]
LD3=pd.DataFrame(LD3)
LD3

,Year,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck,Population
0,2025,1.043706e+06,-70941.417972,17.952350,4.538854,1.089753e+07,2.420311,25.270914,Afghanistan,Asia,0.438159,4.312282e+07
1,2030,1.116381e+06,-89952.464963,18.336933,4.027601,1.212830e+07,2.351515,25.546706,Afghanistan,Asia,0.438159,4.747499e+07
2,2035,1.189056e+06,-108963.511954,18.721517,3.516348,1.335906e+07,2.294273,25.776180,Afghanistan,Asia,0.438159,5.182715e+07
3,2040,1.261731e+06,-127974.558945,19.106101,3.005095,1.458982e+07,2.245900,25.970099,Afghanistan,Asia,0.438159,5.617932e+07
4,2050,1.407081e+06,-165996.652927,19.875268,1.982589,1.705135e+07,2.168622,26.279893,Afghanistan,Asia,0.438159,6.488365e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
1170,2025,1.870439e+05,-148825.326463,18.942374,3.607345,6.033273e+06,1.221004,39.384611,Zimbabwe,Africa,3.104178,1.531886e+07
1171,2030,1.876839e+05,-164660.079143,19.167848,3.451781,6.451871e+06,1.171257,40.263437,Zimbabwe,Africa,3.104178,1.602414e+07
1172,2035,1.883239e+05,-180494.831822,19.393322,3.296216,6.870469e+06,1.125704,41.068164,Zimbabwe,Africa,3.104178,1.672943e+07
1173,2040,1.889639e+05,-196329.584501,19.618796,3.140651,7.289066e+06,1.083837,41.807785,Zimbabwe,Africa,3.104178,1.743471e+07


In [30]:
fig=px.scatter(LD3[LD3['Year']==2050], x="Country", y="LossFunctCheck",range_y=[0,5],
        color="Continent")
fig.update_xaxes(visible=False)
fig.show()

Có vẻ các dự đoán không có vần đề gì nữa.

Tiếp theo ta sử dụng hàm Hồi quy bậc 2 để dự đoán tương lai gần là năm 2022 so với năm 2020.

In [31]:
LD=[i for i in gen_linear(df[df['Year']>=1950],linear_regression_2,[2022])]
LD=pd.DataFrame(LD)
LD=LD.sort_values(by=['Population'],ascending=False).reset_index(drop=True)
LD

,Year,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck,Population
0,2022,3.346121e+06,-411126.352852,40.369599,1.550087,9.384110e+08,0.229571,64.382646,China,Asia,0.091138,1.457553e+09
1,2022,1.296889e+07,-700805.569106,28.761390,1.933698,5.040785e+08,0.901463,35.038327,India,Asia,0.040062,1.438649e+09
2,2022,2.172761e+06,982945.785393,39.282261,2.014710,2.809522e+08,0.641556,82.957421,United States,Northern America,0.268629,3.386704e+08
3,2022,2.806483e+06,-158829.810498,30.976352,2.100638,1.657408e+08,0.999069,59.001419,Indonesia,Asia,0.071430,2.809099e+08
4,2022,4.412810e+06,-294709.132706,23.015787,3.110085,8.112318e+07,1.904715,35.015452,Pakistan,Asia,0.055317,2.316782e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
230,2022,3.853897e+01,0.000000,0.000000,0.000000,3.472436e+02,1.002488,9.032608,Montserrat,Latin America and the Caribbean,0.219118,3.844333e+03
231,2022,1.252865e+02,0.000000,0.000000,0.000000,2.475632e+03,3.603189,71.198181,Falkland Islands,Latin America and the Caribbean,0.085769,3.477100e+03
232,2022,3.929776e+01,0.000000,0.000000,0.000000,7.014603e+02,2.764501,49.346007,Niue,Oceania,0.100013,1.421514e+03
233,2022,2.705358e+01,0.000000,0.000000,0.000000,0.000000e+00,2.197878,0.000000,Tokelau,Oceania,0.091223,1.230895e+03


**Tổng dân số thế giới 2022**

In [32]:
sum_2022 = LD['Population'].sum()
sum_2022

8001369602.050169

Có vẻ là khá sát với thực tế trên trang "https://www.worldometers.info/world-population/" khi dân số 2022 đã **đạt mốc 8 tỷ người**.

**Vậy khi nào thì dân số đạt 9 tỷ người?**

In [33]:
sum_pop = 0
year = 2030
while sum_pop < 9000000000:
    LD=[i for i in gen_linear(df[df['Year']>=1990],linear_regression_2,[year])]
    LD=pd.DataFrame(LD)
    sum_pop = LD['Population'].sum()
    print(str(year)+ ': ' + str(sum_pop))
    year = year + 1
    

2030: 8648797081.485008
2031: 8734648873.204184
2032: 8820645342.809118
2033: 8906786490.299911
2034: 8993072315.676558
2035: 9079502818.93888


Dự đoán năm **2035** dân số sẽ đạt **9 tỷ** người

## **Tổng lại bài toán**

Kết hợp giữa dự đoán tương lai gần và tương lai xa ta đúc kết được mô hình như sau:

In [34]:
LD=[i for i in gen_linear(df[df['Year']>=1990],linear_regression_1,[2025,2030,2035,2040,2050]) ]+[i for i in gen_linear(df[df['Year']>=1950],linear_regression_2,[2022]) ]
LD=pd.DataFrame(LD)
LD=LD.sort_values(by=['Country','Year'],ascending=True).reset_index(drop=True)
LD

,Year,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck,Population
0,2022,1.068893e+06,3977.750578,17.953049,4.730352,1.042072e+07,2.589791,25.248080,Afghanistan,Asia,0.436912,4.127333e+07
1,2025,1.043706e+06,-70941.417972,17.952350,4.538854,1.089753e+07,2.420311,25.270914,Afghanistan,Asia,0.438159,4.312282e+07
2,2030,1.116381e+06,-89952.464963,18.336933,4.027601,1.212830e+07,2.351515,25.546706,Afghanistan,Asia,0.438159,4.747499e+07
3,2035,1.189056e+06,-108963.511954,18.721517,3.516348,1.335906e+07,2.294273,25.776180,Afghanistan,Asia,0.438159,5.182715e+07
4,2040,1.261731e+06,-127974.558945,19.106101,3.005095,1.458982e+07,2.245900,25.970099,Afghanistan,Asia,0.438159,5.617932e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2025,1.870439e+05,-148825.326463,18.942374,3.607345,6.033273e+06,1.221004,39.384611,Zimbabwe,Africa,3.104178,1.531886e+07
1406,2030,1.876839e+05,-164660.079143,19.167848,3.451781,6.451871e+06,1.171257,40.263437,Zimbabwe,Africa,3.104178,1.602414e+07
1407,2035,1.883239e+05,-180494.831822,19.393322,3.296216,6.870469e+06,1.125704,41.068164,Zimbabwe,Africa,3.104178,1.672943e+07
1408,2040,1.889639e+05,-196329.584501,19.618796,3.140651,7.289066e+06,1.083837,41.807785,Zimbabwe,Africa,3.104178,1.743471e+07


In [35]:

LD.to_csv('populationForecast.csv')